In [58]:
import cv2
import numpy as np
import math
import random
import keyboard

import time

In [59]:

def place_car(img, X,Y,voiture):
    hight=math.ceil(voiture.shape[0]/2)
    width=math.ceil(voiture.shape[1]/2)
    if (X-width>=0 and Y-hight>=0 and X+width<img.shape[1] and Y+hight<img.shape[0]) :
        img[Y-hight:Y + hight, X-width:X + width, :] = voiture 
    else:
        if((X-width<0 and X+width>img.shape[1]) or (Y-hight<0 and Y+hight>img.shape[0]) ):
            return img
            
    return img

def remove_car(img, X,Y,voiture):
    game_over=False
    hight=math.ceil(voiture.shape[0]/2)
    width=math.ceil(voiture.shape[1]/2)
    if (X-width>=0 and Y-hight>=0 and X+width<img.shape[1] and Y+hight<img.shape[0]) :
        img[Y-hight:Y + hight, X-width:X + width, :] = np.zeros((voiture.shape[0],voiture.shape[1],3)) 
    return img
def place_meteore(img,liste_meteor):
    for i in liste_meteor:
        img=place_car(img,i.posX,i.posY,i.img)
    return img
def remove_meteore(img,liste_meteor):
    for i in liste_meteor:
        img=remove_car(img,i.posX,i.posY,i.img)
    return img




In [60]:
class Meteore:
    def __init__(self, img,posX,posY,moveX,moveY):
        self.img = img
        self.posX=posX
        self.posY=posY
        self.moveX=moveX
        self.moveY=moveY
        self.radius=int(abs(posX-posY)/2)
    def update(self):
        self.posX+=self.moveX
        self.posY+=self.moveY
    def greet(self):
        print(f"Hello, {self.name}!")
class Voiture:
    def __init__(self, img,posX,posY,speed):
        self.img = img
        self.posX=posX
        self.posY=posY
        self.speed=speed
        self.radius=int(abs(posX-posY)/2)



In [61]:
liste_meteor=[]
for i in range(1):
    met=np.zeros((2*random.randint(10,50),2*random.randint(10,50),3))
    for w in range(met.shape[0]):
        for h in range(met.shape[1]):
            met[w][h][:]=[255,255,255]
    positiony=random.randint(0,300)
    positionx=random.randint(100,540)
    liste_meteor.append(Meteore(met,positionx,positiony,3,5))

In [62]:
for i in liste_meteor:
    print(i.posX,i.posY,i.moveX,i.moveY,i.radius)
    print(i.img.shape)
    print("")
m=liste_meteor.copy()

474 289 3 5 92
(38, 62, 3)



In [74]:
print(m[0].posX,m[0].posY,m[0].moveX,m[0].moveY,m[0].radius)
print(liste_meteor[0].posX,liste_meteor[0].posY,liste_meteor[0].moveX,liste_meteor[0].moveY,liste_meteor[0].radius)

606 650 3 5 92
606 650 3 5 92


In [64]:

voitureimg=np.zeros((100,100,3))
for w in range(voitureimg.shape[0]):
    for h in range(voitureimg.shape[1]):
        voitureimg[w][h][:]=[255,255,255]

voiture=Voiture(voitureimg,320,400,10)




obstaclespeed=100

In [72]:
start=True
game_over=False
cap = cv2.VideoCapture(0)
img=np.zeros(shape=(700, 640, 3))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error reading frame")
        break
    if game_over:
        img=np.zeros(shape=(700, 640, 3))
        cv2.putText(img, "Game Over", (int(640/2)-100, int(700/2)), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)
        cv2.imshow('game', img)
        if cv2.waitKey(1) & 0xFF == ord('a'):
            strart=True
            game_over=False
    else:
        
        if start:
            start=False
            liste_meteor=m
            posx=int(640/2)
            posy=700-70
            voiture.posX=posx
            voiture.posY=posy
            img=np.zeros(shape=(700, 640, 3))
            img=place_car(img,voiture.posX,voiture.posY,voiture.img)
            img=place_meteore(img,liste_meteor)
            cv2.imshow('game', img)
            
            
            
        else:      
            img=remove_car(img,voiture.posX,voiture.posY,voiture.img)
            img=remove_meteore(img,liste_meteor)
            if keyboard.is_pressed('g'):
                voiture.posX-=10
            if keyboard.is_pressed('h'):
                voiture.posX+=10
            
            for i in range(len(liste_meteor)):
                liste_meteor[i].update()
                if liste_meteor[i].posX<0 or liste_meteor[i].posX>img.shape[1]:
                    liste_meteor[i].moveX=-liste_meteor[i].moveX
                if liste_meteor[i].posY<0 or liste_meteor[i].posY>1000:
                    liste_meteor[i].posY=0
            for i in liste_meteor:
                posvm=int((voiture.posX-i.posX)*(voiture.posX-i.posX)+(voiture.posY-i.posY)*(voiture.posY-i.posY))
                posvm=int(math.sqrt(posvm))
                if(posvm<voiture.radius+i.radius):
                    game_over=True
            img=place_car(img,voiture.posX,voiture.posY,voiture.img)
            img=place_meteore(img,liste_meteor)
            cv2.imshow('game', img)
            
            

    # Read the frame from the webcam
    
    
    


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [66]:
cap.release()
cv2.destroyAllWindows()